In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
from PIL import Image
import random
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc, roc_auc_score

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# TensorFlow and Keras
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.regularizers import l2
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils 

# import keras
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics
from tensorflow.keras import optimizers 

print(tf.__version__)



2.1.0


In [2]:
roc_mobilenetv2_SGD_input2 = pd.read_csv('/home/jovyan/scratch-shared/Linnea/Join channels differently/VGG16/SGD/rocauc_VGG16_pretrained_PH_SGD_lr0.01_batch32.csv')
roc_mobilenetv2_Adam_input2 = pd.read_csv('/home/jovyan/scratch-shared/Linnea/Join channels differently/VGG16/Adam lr=0.0001/rocauc_VGG16_pretrained_PH_Adam_lr0.0001_batch32.csv')
#roc_mobilenetv2_np_input2 = pd.read_csv('/home/jovyan/scratch-shared/Linnea/Join channels differently/ResNet50/Not pretrained/rocauc_resnet50_Ph_adam_lr0.0001_batch32_may14.csv')



In [3]:
roc_mobilenetv2_SGD_input2

,Unnamed: 0,target/activity,roc area


In [4]:

sorted_df_sgd = roc_mobilenetv2_SGD_input2.sort_values(by='roc area', ascending=True)
sorted_df_sgd

,Unnamed: 0,target/activity,roc area


In [5]:
sorted_df_adam = roc_mobilenetv2_Adam_input2.sort_values(by='roc area', ascending=True)
sorted_df_adam

,Unnamed: 0,target/activity,roc area
52,72,Rac,0.602633
75,108,"cKit,VEGrFR",0.616687
66,94,"VEGrFR,Src,Raf,Bcr-Abl",0.645917
5,7,"Akt,PDK-1",0.710486
12,17,"CDK,AuroraKinase",0.723963
...,...,...,...
42,58,"PDGFR,cKit,Bcr-Abl",0.998813
16,23,"DUB,Bcr-Abl",0.999152
79,112,"mTOR,PI3K,Akt",0.999866
44,60,"PDK-1,IKK",1.000000


In [6]:
sorted_df_np = roc_mobilenetv2_np_input2.sort_values(by='roc area', ascending=True)
sorted_df_np

,Unnamed: 0,target/activity,roc area
12,17,"CDK,AuroraKinase",0.300089
20,29,FAK,0.322535
42,58,"PDGFR,cKit,Bcr-Abl",0.364951
1,1,AMPK,0.440070
35,50,"JAK,EGFR",0.452439
...,...,...,...
70,100,"c-Met,FGFR,PDGFR",0.963766
62,88,"VEGrFR,Flt",0.966444
74,107,"cKit,Raf,VEGrFR",0.968764
8,10,"AuroraKinase,FGFR,Bcr-Abl,c-RET,Src",0.974074


In [31]:
type(sorted_df_np)

pandas.core.frame.DataFrame

In [39]:
joined_dfs = pd.concat([sorted_df_sgd, sorted_df_adam,sorted_df_np], axis=1, sort=False)
joined_dfs

,Unnamed: 0,target/activity,roc area,Unnamed: 0,target/activity,roc area,Unnamed: 0,target/activity,roc area
0,0,ALK,0.683936,0,ALK,0.929451,0,ALK,0.320303
1,1,AMPK,0.674577,1,AMPK,0.718701,1,AMPK,0.498112
2,2,ATM,0.646680,2,ATM,0.923139,2,ATM,0.582172
3,3,ATM/ATR,0.534081,3,ATM/ATR,0.971763,3,ATM/ATR,0.653612
4,4,Akt,0.655940,4,Akt,0.912251,4,Akt,0.476435
...,...,...,...,...,...,...,...,...,...
76,109,control,0.708636,109,control,0.797087,109,control,0.321163
77,110,mTOR,0.690808,110,mTOR,0.904369,110,mTOR,0.532020
78,111,"mTOR,PI3K",0.618254,111,"mTOR,PI3K",0.925523,111,"mTOR,PI3K",0.127055
79,112,"mTOR,PI3K,Akt",0.825167,112,"mTOR,PI3K,Akt",0.994110,112,"mTOR,PI3K,Akt",0.434360
